Kaggle Competition: https://www.kaggle.com/competitions/asl-fingerspelling

## Exploring Datasets

In [1]:
import pandas as pd
import pyarrow.parquet as pq
import os

In [3]:
#Looking at information within train.csv ans supplemental_metadata.csv
df_train = pd.read_csv('./dataset complete/train.csv')
df_supp_mat = pd.read_csv('./dataset complete/supplemental_metadata.csv')

In [5]:
df_train

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road
...,...,...,...,...,...
67203,train_landmarks/2118949241.parquet,2118949241,388192924,88,431-366-2913
67204,train_landmarks/2118949241.parquet,2118949241,388225542,154,994-392-3850
67205,train_landmarks/2118949241.parquet,2118949241,388232076,95,https://www.tianjiagenomes.com
67206,train_landmarks/2118949241.parquet,2118949241,388235284,36,90 kerwood circle


In [ ]:
len(df_train)

In [ ]:
df_supp_mat.head()

In [ ]:
len(df_supp_mat)

From [here](https://www.kaggle.com/competitions/asl-fingerspelling/discussion/416507https://www.kaggle.com/competitions/asl-fingerspelling/discussion/416507) supplementary material is unnecessary. For now, do not use.

In [ ]:
# read in first parquet file
first_table = pq.read_table('./dataset complete/train_landmarks/105143404.parquet')
first_table = first_table.to_pandas()
first_table

In [ ]:
#We see for each sequence_id, the frames start again from 0
# man kann auch schlussfolgern in dieser datei sind 1000 sätze als vektoren dargestellt
zeros_table = first_table[first_table['frame'] == 0]
zeros_table

In [ ]:
#We see that the sequence_ids  are not in consecutive order the next sequence id after 1784552841 ist 1784574169
first_table[first_table.index == 1784552842]

Sequences are not sequentially numbered.

One sequence corresponds to one sentence.

Frame numberings restart at 0 for each sequence.

One sequence might contain n frames.

In [ ]:
#We print all existinmg columns
for column in first_table.columns.tolist():
    print(column)

<img src="https://camo.githubusercontent.com/7fbec98ddbc1dc4186852d1c29487efd7b1eb820c8b6ef34e113fcde40746be2/68747470733a2f2f6d65646961706970652e6465762f696d616765732f6d6f62696c652f706f73655f747261636b696e675f66756c6c5f626f64795f6c616e646d61726b732e706e67" width="500">
<br>
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHiCz-z3dQ93Zh4p4zq-0dk4A535Dqp5OTHpnqoklaQQ&s" width="500">
<br>
<img src="https://raw.githubusercontent.com/google/mediapipe/a908d668c730da128dfa8d9f6bd25d519d006692/mediapipe/modules/face_geometry/data/canonical_face_model_uv_visualization.png" width="500">

It might make sense to use only hands, mouth/lips, eyes, and eyebrows

In [ ]:
# Using isna() to select all rows with NaN under an entire DataFrame:
df_nan = first_table[first_table.isna().any(axis=1)]
df_nan

In [ ]:
# remove all pose columns except for the arm and wrist landmarks
to_remove = []
for column in first_table.columns.tolist():
    if 'pose' in column:
        if '11' in column:
            continue
        elif '12' in column:
            continue
        elif '13' in column:
            continue
        elif '14' in column:
            continue
        elif '15' in column:
            continue
        elif '16' in column:
            continue
        else:
            to_remove.append(column)

first_table = first_table.drop(to_remove, axis=1)
first_table

In [ ]:
df_nan = first_table[first_table.isna().any(axis=1)]
df_nan

In [ ]:
to_remove = []
for column in first_table.columns.tolist():
    if 'face' in column:
        to_remove.append(column)

first_table = first_table.drop(to_remove, axis=1)

df_nan = first_table[first_table.isna().any(axis=1)]
df_nan

In [ ]:
df_nan.index.unique()

In [ ]:
df_test = df_nan[df_nan.index == 1816603189]
df_test

In [ ]:
len(df_test)

In [ ]:
for column in first_table.columns.tolist():
    print(column)

We have to much nan values even when leaving out pose. Consequently, we need to find a good interpolation scheme.

In [ ]:

files = os.listdir('./dataset complete/train_landmarks')
len(files)

In [ ]:
# interpolation in cas it doesn't work how we want it to work
'''train_landmarks = remove_unwanted_columns(train_landmarks)
# merge the hand and arm data with the corresponding participant id and phrase
df = merge_datasets(train_landmarks, df_train)


phrases = df.sequence_id.unique()
removed_ids = []
df_concat = pd.DataFrame()
INTERPOLATION_NAN_GAP = 30
FILL_NAN_VALUE = -1
SKIP_VALUE = 10000000
pd.options.mode.chained_assignment = None
#for i in 0:
phrase_table = df[df['sequence_id'] == 1784574169]
#nan_count_left_hand = phrase_table['x_left_hand_0'].isna().sum()
#nan_count_right_hand = phrase_table['x_right_hand_0'].isna().sum()
#length_table = len(phrase_table)
#if nan_count_left_hand > 0.75*length_table and nan_count_right_hand > 0.75*length_table:
#    continue
for column in phrase_table.columns.tolist():
    #for column in ['x_left_hand_0', 'x_left_hand_1']:
    if(column == 'sequence_id') or (column == 'frame') or (column == 'participant_id') or (column == 'phrase'):
        continue
    nan_count = phrase_table[column].isnull().astype(int).groupby(phrase_table[column].notnull().cumsum()).cumsum()
    tmp = 0
    for x in range(phrase_table.index[0],phrase_table.index[0] + len(nan_count)-1):
        curr = nan_count[x]
        next = nan_count[x+1]
        if next > curr:
            tmp = next
            if (x == 541):
                if tmp > INTERPOLATION_NAN_GAP:
                    for j in range((x+2)-tmp, x+2):
                        nan_count[j] = SKIP_VALUE
                tmp = 0
        elif next == curr:
            tmp = 0
            continue
        elif (next < curr):
            if tmp > INTERPOLATION_NAN_GAP:
                for j in range(x-tmp, x+1):
                    nan_count[j] = SKIP_VALUE
            tmp = 0
    nan_count = nan_count.sort_index()
    phrase_table[column] = phrase_table[column].interpolate() \
        .where(nan_count < SKIP_VALUE, np.nan)



phrase_table = phrase_table.fillna(FILL_NAN_VALUE)
df_concat = pd.concat([df_concat, phrase_table], axis = 0)
df_concat'''

# Preprocessing TODOs
- goal: load first 30 parquet files
- for each parquet file select only relevant columns (remove face date and pose date except for arms)
- for each file remove NaNs (replace with -1 or interpolate if previous and following values are known)
- add participant number and label phrase to parquet dfs
- transform label phrase to array/string of numbers
- combine dfs
- export as one or two (hopefully not so) large pickle file

Tips:
- For each parquet file do preprocessing and save as pickle
- later combine multiple pickle files to one file
- to do preprocessing faster write preprocessing steps as function to only change name of file


Next build a Transformer.

# Preprocessing
## Train_Landmarks
Removing unnecessary columns

In [47]:
def remove_unwanted_columns(df):
    # remove all pose columns except for the arm and wrist landmarks
    to_remove_pose = []
    for column in df.columns.tolist():
        if 'pose' in column:
            if '11' in column:
                continue
            elif '12' in column:
                continue
            elif '13' in column:
                continue
            elif '14' in column:
                continue
            elif '15' in column:
                continue
            elif '16' in column:
                continue
            else:
                to_remove_pose.append(column)

    df = df.drop(to_remove_pose, axis=1)


    to_remove_face = []
    for column in df.columns.tolist():
        if 'face' in column:
            to_remove_face.append(column)

    df = df.drop(to_remove_face, axis=1)

    return df

## Edit train_csv
change Strings to number arrays

In [48]:
def exchange_string_with_numbers_array(df, characters):
    for i in range(0, len(df)):
        num_array = [characters.get(char, char) for char in df.iloc[i]['phrase']]
        df.at[i, 'phrase'] = num_array
    columns = ['sequence_id', 'participant_id', 'phrase']
    df_train_selected = df[columns]
    return df_train_selected

## Merge datasets
merge hand and arm data with participant id and phrase

In [49]:
def merge_datasets(df, df_phrases):
    df_merged = pd.merge(df, df_phrases, on='sequence_id', how = 'left')
    return df_merged

## Interpolation of nan values
interpolate for nan gaps shorter than 30 following directly, else replace nan values with impossible number value (= out of frame)
This happens for each phrase dataset so we don't interpolate over multiple sentences

In [73]:
#trivial interpolation function
INTERPOLATION_NAN_GAP = 15
FILL_NAN_VALUE = -1
SKIP_VALUE = 10000000
pd.options.mode.chained_assignment = None
file_counter = 100000
test_participant_ids = [1, 38, 202, 71, 74, 102, 109, 125, 225, 161, 242, 59, 36, 188, 154, 80, 113, 88, 68]
train_participant_ids = [73, 95, 147, 18, 115, 216, 168, 56, 21, 159, 141, 143, 233, 181, 236, 9, 196, 53, 254, 251, 157, 20, 203, 112, 15, 72, 27, 178, 6, 153, 92, 169, 10, 145, 24, 192, 230, 186, 2, 135, 246, 43, 93, 33, 223, 81, 160, 231, 219, 171, 136, 25, 40, 176, 158, 121, 63, 105, 4, 128, 89, 13, 70, 138, 187, 241, 217, 107, 227, 122, 151, 117, 239, 0, 76]


def interpolate_dataset(df):
    global file_counter
    phrases = df.sequence_id.unique()
    df_concat = pd.DataFrame()
    for i in phrases:
        phrase_table = df[df['sequence_id'] == i]
        nan_count_left_hand = phrase_table['x_left_hand_0'].isna().sum()
        nan_count_right_hand = phrase_table['x_right_hand_0'].isna().sum()
        length_table = len(phrase_table)
        if nan_count_left_hand > 0.75*length_table and nan_count_right_hand > 0.75*length_table:
            continue
        for column in phrase_table.columns.tolist():
            if(column == 'sequence_id') or (column == 'frame') or (column == 'participant_id') or (column == 'phrase'):
                continue
            nan_count = phrase_table[column].isnull().astype(int).groupby(phrase_table[column].notnull().cumsum()).cumsum()
            tmp = 0
            for x in range(phrase_table.index[0],phrase_table.index[0] + len(nan_count)-1):
                curr = nan_count[x]
                next = nan_count[x+1]
                if next > curr:
                    tmp = next
                    if (x == phrase_table.index[0] + len(nan_count)-2):
                        for j in range(((x+2)-tmp), x+2):
                            nan_count[j] = SKIP_VALUE
                elif next == curr:
                    tmp = 0
                    continue
                elif (next < curr):
                    if tmp > INTERPOLATION_NAN_GAP:
                        for j in range(x-tmp, x+1):
                            nan_count[j] = SKIP_VALUE
                    tmp = 0
            nan_count = nan_count.sort_index()
            phrase_table[column] = phrase_table[column].interpolate() \
                .where(nan_count < SKIP_VALUE, np.nan)

        phrase_table = phrase_table.fillna(FILL_NAN_VALUE)

                #df_concat = pd.concat([df_concat, phrase_table], axis = 0)
        file_counter += 1

        if phrase_table.participant_id.unique()[0] in test_participant_ids:
            file_path = './dataset complete/preprocessed_files_data_generator/test_ones/' + str(file_counter) + '.pkl'
            phrase_table.to_pickle(file_path)
        else:
            file_path = './dataset complete/preprocessed_files_data_generator/train_ones/' + str(file_counter) + '.pkl'
            phrase_table.to_pickle(file_path)





# Create preprocessed Files


In [51]:
import pandas as pd
import pyarrow.parquet as pq
import os
import json
import numpy as np
import pickle

In [63]:
df_train = pd.read_csv('./dataset complete/train.csv')

with open ("./dataset complete/character_to_prediction_index.json", "r") as f:
    characters = json.load(f)

df_train = exchange_string_with_numbers_array(df_train, characters)
df_train

,sequence_id,participant_id,phrase
0,1816796431,217,"[18, 0, 34, 49, 36, 36, 42, 39, 46, 52, 50, 36]"
1,1816825349,107,"[50, 34, 32, 43, 36, 50, 14, 42, 52, 39, 32, 5..."
2,1816909464,1,"[16, 18, 23, 18, 0, 54, 40, 43, 43, 40, 32, 44..."
3,1816967051,63,"[24, 23, 23, 0, 37, 49, 32, 45, 42, 43, 40, 45..."
4,1817123330,89,"[21, 24, 17, 15, 0, 45, 46, 49, 51, 39, 36, 32..."
...,...,...,...
67203,388192924,88,"[19, 18, 16, 12, 18, 21, 21, 12, 17, 24, 16, 18]"
67204,388225542,154,"[24, 24, 19, 12, 18, 24, 17, 12, 18, 23, 20, 15]"
67205,388232076,95,"[39, 51, 51, 47, 50, 25, 14, 14, 54, 54, 54, 1..."
67206,388235284,36,"[24, 15, 0, 42, 36, 49, 54, 46, 46, 35, 0, 34,..."


In [70]:
supp_train = pd.read_csv('./dataset complete/supplemental_metadata.csv')

with open ("./dataset complete/character_to_prediction_index.json", "r") as f:
    characters = json.load(f)

supp_train = exchange_string_with_numbers_array(supp_train, characters)
supp_train

,sequence_id,participant_id,phrase
0,1535467051,251,"[34, 46, 44, 40, 45, 38, 0, 52, 47, 0, 54, 40,..."
1,1535499058,239,"[54, 36, 0, 33, 36, 51, 51, 36, 49, 0, 40, 45,..."
2,1535530550,245,"[40, 45, 51, 36, 49, 36, 50, 51, 40, 45, 38, 0..."
3,1535545499,38,"[53, 40, 34, 51, 40, 44, 50, 0, 35, 36, 50, 36..."
4,1535585216,254,"[42, 45, 36, 36, 0, 33, 46, 45, 36, 0, 40, 50,..."
...,...,...,...
52953,1090866442,239,"[54, 32, 45, 51, 0, 51, 46, 0, 41, 46, 40, 45,..."
52954,1090966452,95,"[51, 39, 40, 50, 0, 47, 39, 36, 45, 46, 44, 36..."
52955,1091005846,40,"[51, 39, 36, 0, 54, 40, 45, 45, 36, 49, 0, 46,..."
52956,1091011550,241,"[32, 49, 36, 0, 56, 46, 52, 0, 50, 52, 49, 36,..."


In [24]:
supp_ids = supp_train.participant_id.unique()
df_supp_ids = pd.DataFrame({'IDs': supp_ids})
df_supp_ids

,IDs
0,251
1,239
2,245
3,38
4,254
...,...
67,150
68,63
69,56
70,51


In [ ]:
# read in first parquet file
train_landmarks = pq.read_table('./dataset complete/train_landmarks/105143404.parquet')
train_landmarks = train_landmarks.to_pandas()
train_landmarks

In [ ]:
# leave only columns with hand and arm data
train_landmarks = remove_unwanted_columns(train_landmarks)
# merge the hand and arm data with the corresponding participant id and phrase
df = merge_datasets(train_landmarks, df_train)
# for short nan gaps interpolate the values, else fill Nans with a placeholder number
df = interpolate_dataset(df)
# check if all nans have been removed
if len(df[df.isna().any(axis=1)]) != 0:
    print("Still NANs!")
# print df
df

In [2]:
import os
import shutil
import pickle
import pandas as pd
source = './dataset complete/preprocessed_files_data_generator/train_ones'
destination = './dataset complete/preprocessed_files_data_generator/val_ones'

for filename in os.listdir(destination):
    filepath = os.path.join(destination, filename)
    destinationpath = os.path.join(source, filename)
    shutil.move(filepath, destinationpath)

In [1]:
import os
import shutil
import pickle
import pandas as pd

val_participant_ids = [21, 10]

source = './dataset complete/preprocessed_files_data_generator/train_ones'
destination = './dataset complete/preprocessed_files_data_generator/val_ones'

for filename in os.listdir(destination):
    filepath = os.path.join(destination, filename)
    with open(filepath, "rb") as file:
        df = pickle.load(file)
    if df.iloc[0]['participant_id'] in val_participant_ids:
        destinationpath = os.path.join(source, filename)
        shutil.move(filepath, destinationpath)
    else:
        continue


In [4]:
with open('./dataset complete/preprocessed_files_data_generator/val_ones/5.pkl', "rb") as file:
    df = pickle.load(file)
df

,sequence_id,frame,x_left_hand_0,x_left_hand_1,x_left_hand_2,x_left_hand_3,x_left_hand_4,x_left_hand_5,x_left_hand_6,x_left_hand_7,...,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20,participant_id,phrase
1583,1975645112,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.006669,-0.038562,-0.033234,-0.021116,-0.013610,-0.037306,-0.029544,-0.016587,89,"[10, 18, 24, 12, 21, 16, 18, 19, 12, 19, 21, 2..."
1584,1975645112,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.011597,-0.045103,-0.039606,-0.027220,-0.018785,-0.043498,-0.035576,-0.022391,89,"[10, 18, 24, 12, 21, 16, 18, 19, 12, 19, 21, 2..."
1585,1975645112,2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.016526,-0.051644,-0.045977,-0.033324,-0.023961,-0.049691,-0.041609,-0.028194,89,"[10, 18, 24, 12, 21, 16, 18, 19, 12, 19, 21, 2..."
1586,1975645112,3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.021454,-0.058185,-0.052348,-0.039428,-0.029136,-0.055884,-0.047641,-0.033998,89,"[10, 18, 24, 12, 21, 16, 18, 19, 12, 19, 21, 2..."
1587,1975645112,4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.026382,-0.064725,-0.058720,-0.045532,-0.034312,-0.062077,-0.053673,-0.039801,89,"[10, 18, 24, 12, 21, 16, 18, 19, 12, 19, 21, 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,1975645112,307,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.060169,-0.122437,-0.120431,-0.100585,-0.072385,-0.113016,-0.114382,-0.101061,89,"[10, 18, 24, 12, 21, 16, 18, 19, 12, 19, 21, 2..."
1891,1975645112,308,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.055383,-0.115317,-0.114304,-0.096927,-0.066630,-0.107399,-0.106817,-0.093154,89,"[10, 18, 24, 12, 21, 16, 18, 19, 12, 19, 21, 2..."
1892,1975645112,309,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.057277,-0.119451,-0.118618,-0.100871,-0.068040,-0.110561,-0.110304,-0.096480,89,"[10, 18, 24, 12, 21, 16, 18, 19, 12, 19, 21, 2..."
1893,1975645112,310,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.044779,-0.102835,-0.099473,-0.079704,-0.055282,-0.095526,-0.094276,-0.079275,89,"[10, 18, 24, 12, 21, 16, 18, 19, 12, 19, 21, 2..."


Save as Pickle file:

In [ ]:
#file_path = './dataset complete/preprocessed_files/105143404.pkl'
#df.to_pickle(file_path)

# Creating preprocessed data files

In [64]:
file_names = os.listdir('./dataset complete/train_landmarks/')
for i in range(0, len(file_names)):
    file_names[i] = file_names[i].replace('.parquet', '')

file_names

['1019715464',
 '1021040628',
 '105143404',
 '1098899348',
 '1099408314',
 '1133664520',
 '1134756332',
 '1255240050',
 '128822441',
 '1320204318',
 '1341528257',
 '1358493307',
 '1365275733',
 '1365772051',
 '1405046009',
 '1448136004',
 '1497621680',
 '149822653',
 '152029243',
 '1552432300',
 '1557244878',
 '1562234637',
 '1643479812',
 '1647220008',
 '1662742697',
 '1664666588',
 '169560558',
 '1726141437',
 '175396851',
 '1785039512',
 '1865557033',
 '1880177496',
 '1905462118',
 '1906357076',
 '1920330615',
 '1967755728',
 '1969985709',
 '1997878546',
 '2026717426',
 '2036580525',
 '2072296290',
 '2072876091',
 '2118949241',
 '234418913',
 '296317215',
 '349393104',
 '388576474',
 '425182931',
 '433948159',
 '450474571',
 '474255203',
 '495378749',
 '522550314',
 '527708222',
 '532011803',
 '5414471',
 '546816846',
 '566963657',
 '568753759',
 '614661748',
 '638508439',
 '649779897',
 '654436541',
 '683666742',
 '871280215',
 '882979387',
 '933868835',
 '939623093']

In [65]:
for file in file_names:
    print(file)

    # read in first parquet file
    train_landmarks = pq.read_table('./dataset complete/train_landmarks/' + file + '.parquet')
    train_landmarks = train_landmarks.to_pandas()

    # leave only columns with hand and arm data
    train_landmarks = remove_unwanted_columns(train_landmarks)
    # merge the hand and arm data with the corresponding participant id and phrase
    df = merge_datasets(train_landmarks, df_train)
    # for short nan gaps interpolate the values, else fill Nans with a placeholder number
    interpolate_dataset(df)

1019715464
1021040628
105143404
1098899348
1099408314
1133664520
1134756332
1255240050
128822441
1320204318
1341528257
1358493307
1365275733
1365772051
1405046009
1448136004
1497621680
149822653
152029243
1552432300
1557244878
1562234637
1643479812
1647220008
1662742697
1664666588
169560558
1726141437
175396851
1785039512
1865557033
1880177496
1905462118
1906357076
1920330615
1967755728
1969985709
1997878546
2026717426
2036580525
2072296290
2072876091
2118949241
234418913
296317215
349393104
388576474
425182931
433948159
450474571
474255203
495378749
522550314
527708222
532011803
5414471
546816846
566963657
568753759
614661748
638508439
649779897
654436541
683666742
871280215
882979387
933868835
939623093


In [74]:
file_names = os.listdir('./dataset complete/supplemental_landmarks/')
for i in range(0, len(file_names)):
    file_names[i] = file_names[i].replace('.parquet', '')

file_names

['1032110484',
 '1047404576',
 '1112747136',
 '1118603411',
 '1144115867',
 '1176508147',
 '1249944812',
 '1279694894',
 '131312512',
 '1407656790',
 '1471096258',
 '1471341722',
 '1505488209',
 '1579345709',
 '1624527344',
 '1650637630',
 '1682915129',
 '1727438550',
 '1755047076',
 '1756773911',
 '1779786322',
 '1857374937',
 '1881515495',
 '193950599',
 '2057261717',
 '2100073719',
 '236903981',
 '285528514',
 '293101677',
 '333606065',
 '33432165',
 '369584223',
 '371169664',
 '440362090',
 '442061898',
 '471766624',
 '595441814',
 '597469033',
 '636900267',
 '639454452',
 '676340265',
 '680303484',
 '697480828',
 '716508881',
 '736978972',
 '756566775',
 '775880548',
 '778903889',
 '86446671',
 '897287709',
 '924144755',
 '95345213',
 '971104021']

In [75]:
for file in file_names:
    print(file)

    # read in first parquet file
    train_landmarks = pq.read_table('./dataset complete/supplemental_landmarks/' + file + '.parquet')
    train_landmarks = train_landmarks.to_pandas()

    # leave only columns with hand and arm data
    train_landmarks = remove_unwanted_columns(train_landmarks)
    # merge the hand and arm data with the corresponding participant id and phrase
    df = merge_datasets(train_landmarks, supp_train)
    # for short nan gaps interpolate the values, else fill Nans with a placeholder number
    df = interpolate_dataset(df)

1032110484
1047404576
1112747136
1118603411
1144115867
1176508147
1249944812
1279694894
131312512
1407656790
1471096258
1471341722
1505488209
1579345709
1624527344
1650637630
1682915129
1727438550
1755047076
1756773911
1779786322
1857374937
1881515495
193950599
2057261717
2100073719
236903981
285528514
293101677
333606065
33432165
369584223
371169664
440362090
442061898
471766624
595441814
597469033
636900267
639454452
676340265
680303484
697480828
716508881
736978972
756566775
775880548
778903889
86446671
897287709
924144755
95345213
971104021


Read in pickle files:

In [ ]:
#df_pkl = pd.read_pickle('./dataset complete/preprocessed_files/933868835.pkl')
#print(len(df_pkl))
#df_pkl

In [ ]:
# Save as csv to have a look if dataset is preprocessed like we wanted
#df_pkl.to_csv('./dataset complete/preprocessed_files/test.csv')
#import os
#path = './dataset complete/preprocessed_files/test.csv'
#os.remove(path)

Merge pickle files

In [76]:
df1 = pd.read_pickle('./dataset complete/preprocessed_files_data_generator/train_ones/100001.pkl')
#df2 = pd.read_pickle('./dataset complete/preprocessed_files/105143404.pkl')
#df = pd.concat([df1, df2], axis=0)
df1

,sequence_id,frame,x_left_hand_0,x_left_hand_1,x_left_hand_2,x_left_hand_3,x_left_hand_4,x_left_hand_5,x_left_hand_6,x_left_hand_7,...,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20,participant_id,phrase
14,1617965404,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.179770,-0.256494,-0.242231,-0.220745,-0.199080,-0.243315,-0.227379,-0.207144,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
15,1617965404,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.133887,-0.191751,-0.176973,-0.157588,-0.147698,-0.182360,-0.169217,-0.153210,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
16,1617965404,2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.147321,-0.210993,-0.192886,-0.169733,-0.161708,-0.201847,-0.186980,-0.168545,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
17,1617965404,3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.160755,-0.230234,-0.208799,-0.181878,-0.175718,-0.221333,-0.204742,-0.183880,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
18,1617965404,4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.174188,-0.249476,-0.224711,-0.194022,-0.189728,-0.240820,-0.222504,-0.199214,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,1617965404,184,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.160375,-0.243911,-0.239989,-0.220944,-0.164556,-0.220082,-0.219227,-0.208376,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
199,1617965404,185,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.147414,-0.230755,-0.225177,-0.203669,-0.175644,-0.235811,-0.233191,-0.218580,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
200,1617965404,186,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.095521,-0.170518,-0.163314,-0.138840,-0.118960,-0.176716,-0.173925,-0.157661,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
201,1617965404,187,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.151411,-0.236903,-0.228139,-0.204463,-0.178663,-0.241084,-0.236646,-0.220116,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."


# Transformer

In [ ]:
import json

with open ("./dataset complete/character_to_prediction_index.json", "r") as f:
    characters = json.load(f)
len(characters)

In [ ]:
from transformer import get_compiled_transformer

# TODO: tune hyperparameters
d_model = 128  # hidden layer(s) dimensionality
num_layers = 5 # how many encoders and decoders to stack
num_heads = 8 # how many attention heads should every mha have
ff_dim = 256 # how many neurons shall feed-forward layers have
dropout_rate = 0.1

output_vocab_size = len(characters) + 2

transformer = get_compiled_transformer(
    d_model=d_model,
    num_layers=num_layers,
    num_heads=num_heads,
    ff_dim=ff_dim,
    dropout_rate=dropout_rate,
    output_vocab_size=output_vocab_size
)

In [2]:
import pandas as pd
df_data = pd.read_pickle('./dataset complete/preprocessed_files/532011803.pkl')
df_data

,sequence_id,frame,x_left_hand_0,x_left_hand_1,x_left_hand_2,x_left_hand_3,x_left_hand_4,x_left_hand_5,x_left_hand_6,x_left_hand_7,...,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20,participant_id,phrase
0,419070075,0,0.736679,0.652913,0.591140,0.566396,0.554301,0.570637,0.520308,0.491068,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,154,"[17, 23, 15, 16, 0, 36, 51, 51, 36, 49, 50]"
1,419070075,1,0.727973,0.638127,0.578803,0.547968,0.534888,0.540625,0.486297,0.450276,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,154,"[17, 23, 15, 16, 0, 36, 51, 51, 36, 49, 50]"
2,419070075,2,0.764039,0.661006,0.584672,0.548870,0.535731,0.559617,0.524162,0.502584,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,154,"[17, 23, 15, 16, 0, 36, 51, 51, 36, 49, 50]"
3,419070075,3,0.760813,0.658978,0.582197,0.544619,0.528777,0.558918,0.519521,0.495088,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,154,"[17, 23, 15, 16, 0, 36, 51, 51, 36, 49, 50]"
4,419070075,4,0.757586,0.656950,0.579722,0.540368,0.521822,0.558219,0.514880,0.487591,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,154,"[17, 23, 15, 16, 0, 36, 51, 51, 36, 49, 50]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152859,450190041,222,0.847533,0.823417,0.749445,0.672636,0.604092,0.756974,0.686968,0.645792,...,-0.060506,-0.104001,-0.140276,-0.157513,-0.075748,-0.110919,-0.131616,-0.140019,53,"[54, 40, 42, 47, 46, 43, 12, 45, 50, 13, 47, 4..."
152860,450190041,223,0.847533,0.823417,0.749445,0.672636,0.604092,0.756974,0.686968,0.645792,...,-0.059903,-0.103407,-0.137230,-0.151857,-0.076443,-0.112238,-0.131055,-0.137239,53,"[54, 40, 42, 47, 46, 43, 12, 45, 50, 13, 47, 4..."
152861,450190041,224,0.847533,0.823417,0.749445,0.672636,0.604092,0.756974,0.686968,0.645792,...,-0.086146,-0.128921,-0.162396,-0.178761,-0.105032,-0.138215,-0.154958,-0.161438,53,"[54, 40, 42, 47, 46, 43, 12, 45, 50, 13, 47, 4..."
152862,450190041,225,0.847533,0.823417,0.749445,0.672636,0.604092,0.756974,0.686968,0.645792,...,-0.116743,-0.166796,-0.187481,-0.193131,-0.139439,-0.176661,-0.183826,-0.181551,53,"[54, 40, 42, 47, 46, 43, 12, 45, 50, 13, 47, 4..."


In [ ]:
# TODO: Tune - the longer the worse is the performance
MAX_PHRASE_LENGTH = 100
MAX_SIGN_LENGTH = 800

In [ ]:
from copy import deepcopy
import numpy as np

phrases = df_data.sequence_id.unique()
columns = ['sequence_id', 'frame', 'participant_id', 'phrase']
targets = []
inputs = []
contexts = []
for i in phrases:
    phrase_table = df_data[df_data['sequence_id'] == i]
    phrase = phrase_table.iloc[0].phrase
    if len(phrase) > MAX_PHRASE_LENGTH:
        print('Houston we have a problem!')
    while len(phrase) < MAX_PHRASE_LENGTH:
        phrase.append(59)
    context = deepcopy(phrase)
    context.insert(0, 60)
    contexts.append(context[:-1])
    targets.append(phrase)

    phrase_table = phrase_table.drop(columns, axis=1)
    phrase_table = phrase_table.to_numpy(copy=True)

    if phrase_table.shape[0] > MAX_SIGN_LENGTH:
        print('Houuuuuussssteeeeeen!')

    phrase_table = np.pad(phrase_table, [(0, MAX_SIGN_LENGTH-phrase_table.shape[0]), (0,0)], 'constant', constant_values=(-1, -1))

    inputs.append(phrase_table)

inputs = np.stack(inputs)
contexts = np.array(contexts)
targets = np.array(targets)
print(inputs.shape)
print(targets.shape)
print(contexts.shape)

In [ ]:
EPOCHS = 5
BATCH_SIZE=64

inputs = [inputs, contexts]

transformer.fit(inputs, targets, epochs=EPOCHS, batch_size=BATCH_SIZE)

In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [1]:
import sys
print(sys.version)

3.9.16 (main, Jan 11 2023, 16:16:36) [MSC v.1916 64 bit (AMD64)]


# Train Test Split

In [1]:
import pandas as pd

df_test = pd.read_csv('./dataset complete/train.csv')
df_test

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road
...,...,...,...,...,...
67203,train_landmarks/2118949241.parquet,2118949241,388192924,88,431-366-2913
67204,train_landmarks/2118949241.parquet,2118949241,388225542,154,994-392-3850
67205,train_landmarks/2118949241.parquet,2118949241,388232076,95,https://www.tianjiagenomes.com
67206,train_landmarks/2118949241.parquet,2118949241,388235284,36,90 kerwood circle


In [2]:
p_ids = df_test.participant_id.unique().tolist()
len(p_ids)

94

In [3]:
import random
TRAIN_SPLIT = 0.8
TEST_SPLIT = 0.2

num_train_samples = int(TRAIN_SPLIT * len(p_ids))

train_participants = random.sample(p_ids, num_train_samples)
test_participants = [x for x in p_ids if x not in train_participants]

In [4]:
from copy import deepcopy
import numpy as np
from tqdm import tqdm
import os

# TODO: Tune - the longer the worse is the performance
MAX_PHRASE_LENGTH = 100
MAX_SIGN_LENGTH = 900

columns = ['sequence_id', 'frame', 'participant_id', 'phrase']
targets_train = []
inputs_train = []
contexts_train = []
targets_test = []
inputs_test = []
contexts_test = []

files = os.listdir('./dataset complete/preprocessed_files')

for file in tqdm(files):
    df_data = pd.read_pickle('./dataset complete/preprocessed_files/' + file)
    phrases = df_data.sequence_id.unique()

    for i in phrases:
        phrase_table = df_data[df_data['sequence_id'] == i]

        is_test = False
        if phrase_table.participant_id.unique()[0] in test_participants:
            is_test = True

        phrase = phrase_table.iloc[0].phrase
        if len(phrase) > MAX_PHRASE_LENGTH:
            print('Houston we have a problem!')
        while len(phrase) < MAX_PHRASE_LENGTH:
            phrase.append(59)
        context = deepcopy(phrase)
        context.insert(0, 60)

        phrase_table = phrase_table.drop(columns, axis=1)
        phrase_table = phrase_table.to_numpy(copy=True)

        if phrase_table.shape[0] > MAX_SIGN_LENGTH:
            print('Houuuuuussssteeeeeen!')

        # TODO: pad with last known value not -1
        phrase_table = np.pad(phrase_table, [(0, MAX_SIGN_LENGTH-phrase_table.shape[0]), (0,0)], 'constant', constant_values=(-1, -1))

        if not is_test:
            inputs_train.append(phrase_table)
            targets_train.append(phrase)
            contexts_train.append(context[:-1])
        else:
            inputs_test.append(phrase_table)
            targets_test.append(phrase)
            contexts_test.append(context[:-1])

inputs_train = np.stack(inputs_train)
contexts_train = np.array(contexts_train)
targets_train = np.array(targets_train)
inputs_test = np.stack(inputs_test)
contexts_test = np.array(contexts_test)
targets_test = np.array(targets_test)

100%|██████████| 68/68 [01:09<00:00,  1.02s/it]


In [5]:
print(inputs_train.shape)
print(targets_train.shape)
print(contexts_train.shape)

(43110, 900, 144)
(43110, 100)
(43110, 100)


In [6]:
print(inputs_test.shape)
print(targets_test.shape)
print(contexts_test.shape)

(10143, 900, 144)
(10143, 100)
(10143, 100)


In [7]:
import pickle

with open('./dataset complete/train_test_data/train_data.pkl', 'wb') as file:
    train_data = [(inputs_train, contexts_train), targets_train]
    pickle.dump(train_data, file)

with open('./dataset complete/train_test_data/test_data.pkl', 'wb') as file:
    test_data = [(inputs_test, contexts_test), targets_test]
    pickle.dump(test_data, file)

# New preprocessing

In [12]:
import pandas as pd
import pyarrow.parquet as pq
import os
import json
import numpy as np
import pickle

In [6]:
df_train = pd.read_csv('./dataset complete/train.csv')

with open ("./dataset complete/character_to_prediction_index.json", "r") as f:
    characters = json.load(f)

df_train = exchange_string_with_numbers_array(df_train, characters)
df_train

,sequence_id,participant_id,phrase
0,1816796431,217,"[18, 0, 34, 49, 36, 36, 42, 39, 46, 52, 50, 36]"
1,1816825349,107,"[50, 34, 32, 43, 36, 50, 14, 42, 52, 39, 32, 5..."
2,1816909464,1,"[16, 18, 23, 18, 0, 54, 40, 43, 43, 40, 32, 44..."
3,1816967051,63,"[24, 23, 23, 0, 37, 49, 32, 45, 42, 43, 40, 45..."
4,1817123330,89,"[21, 24, 17, 15, 0, 45, 46, 49, 51, 39, 36, 32..."
...,...,...,...
67203,388192924,88,"[19, 18, 16, 12, 18, 21, 21, 12, 17, 24, 16, 18]"
67204,388225542,154,"[24, 24, 19, 12, 18, 24, 17, 12, 18, 23, 20, 15]"
67205,388232076,95,"[39, 51, 51, 47, 50, 25, 14, 14, 54, 54, 54, 1..."
67206,388235284,36,"[24, 15, 0, 42, 36, 49, 54, 46, 46, 35, 0, 34,..."


In [7]:
import os
file_names = os.listdir('./dataset complete/train_landmarks/')

file_names

['1019715464.parquet',
 '1021040628.parquet',
 '105143404.parquet',
 '1098899348.parquet',
 '1099408314.parquet',
 '1133664520.parquet',
 '1134756332.parquet',
 '1255240050.parquet',
 '128822441.parquet',
 '1320204318.parquet',
 '1341528257.parquet',
 '1358493307.parquet',
 '1365275733.parquet',
 '1365772051.parquet',
 '1405046009.parquet',
 '1448136004.parquet',
 '1497621680.parquet',
 '149822653.parquet',
 '152029243.parquet',
 '1552432300.parquet',
 '1557244878.parquet',
 '1562234637.parquet',
 '1643479812.parquet',
 '1647220008.parquet',
 '1662742697.parquet',
 '1664666588.parquet',
 '169560558.parquet',
 '1726141437.parquet',
 '175396851.parquet',
 '1785039512.parquet',
 '1865557033.parquet',
 '1880177496.parquet',
 '1905462118.parquet',
 '1906357076.parquet',
 '1920330615.parquet',
 '1967755728.parquet',
 '1969985709.parquet',
 '1997878546.parquet',
 '2026717426.parquet',
 '2036580525.parquet',
 '2072296290.parquet',
 '2072876091.parquet',
 '2118949241.parquet',
 '234418913.parqu

In [9]:
import pyarrow.parquet as pq
df_concat = pd.DataFrame()

for file in file_names:
    print(file)
    train_landmarks = pq.read_table('./dataset complete/train_landmarks/' + file)
    train_landmarks = train_landmarks.to_pandas()
    train_landmarks = remove_unwanted_columns(train_landmarks)
    # merge the hand and arm data with the corresponding participant id and phrase
    df = merge_datasets(train_landmarks, df_train)
    df_concat = pd.concat([df_concat, df], axis = 0)

1019715464.parquet
1021040628.parquet
105143404.parquet
1098899348.parquet
1099408314.parquet
1133664520.parquet
1134756332.parquet
1255240050.parquet
128822441.parquet
1320204318.parquet
1341528257.parquet
1358493307.parquet
1365275733.parquet
1365772051.parquet
1405046009.parquet
1448136004.parquet
1497621680.parquet
149822653.parquet
152029243.parquet
1552432300.parquet
1557244878.parquet
1562234637.parquet


KeyboardInterrupt: 

In [8]:
mean_columns = dict()

for column in df_concat.columns.tolist():
    if(column == 'sequence_id') or (column == 'frame') or (column == 'participant_id') or (column == 'phrase'):
        continue
    mean_columns[column] = float(df_concat[column].mean(skipna=True))


mean_columns

NameError: name 'df_concat' is not defined

In [23]:
with open('./dataset complete/means_columns.json', 'w') as fp:
    json.dump(mean_columns, fp)

## New Interpolation

In [18]:
train_landmarks = pq.read_table('./dataset complete/train_landmarks/128822441.parquet')
train_landmarks = train_landmarks.to_pandas()
train_landmarks = remove_unwanted_columns(train_landmarks)
# merge the hand and arm data with the corresponding participant id and phrase
df = merge_datasets(train_landmarks, df_train)
df

,sequence_id,frame,x_left_hand_0,x_left_hand_1,x_left_hand_2,x_left_hand_3,x_left_hand_4,x_left_hand_5,x_left_hand_6,x_left_hand_7,...,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20,participant_id,phrase
0,197983539,0,0.744336,0.629080,0.501370,0.406846,0.350579,0.463368,0.346668,0.282930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178,"[17, 23, 19, 0, 47, 32, 49, 42, 50, 0, 32, 45,..."
1,197983539,1,0.690452,0.570185,0.445545,0.359518,0.301777,0.418598,0.301317,0.243037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178,"[17, 23, 19, 0, 47, 32, 49, 42, 50, 0, 32, 45,..."
2,197983539,2,0.715965,0.590581,0.463443,0.375573,0.320727,0.410953,0.302580,0.234283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178,"[17, 23, 19, 0, 47, 32, 49, 42, 50, 0, 32, 45,..."
3,197983539,3,0.724584,0.607585,0.482424,0.392845,0.333539,0.429988,0.313290,0.239941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178,"[17, 23, 19, 0, 47, 32, 49, 42, 50, 0, 32, 45,..."
4,197983539,4,0.749360,0.617729,0.488442,0.395161,0.329791,0.448935,0.330655,0.252471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178,"[17, 23, 19, 0, 47, 32, 49, 42, 50, 0, 32, 45,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157495,228449435,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121,"[20, 16, 21, 20, 0, 33, 49, 46, 54, 45, 0, 47,..."
157496,228449435,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121,"[20, 16, 21, 20, 0, 33, 49, 46, 54, 45, 0, 47,..."
157497,228449435,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121,"[20, 16, 21, 20, 0, 33, 49, 46, 54, 45, 0, 47,..."
157498,228449435,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121,"[20, 16, 21, 20, 0, 33, 49, 46, 54, 45, 0, 47,..."


In [2]:
import pickle
with open('./dataset complete/preprocessed_files_data_generator/train_ones/4.pkl', 'rb') as file:
    df = pickle.load(file)
df

,sequence_id,frame,x_left_hand_0,x_left_hand_1,x_left_hand_2,x_left_hand_3,x_left_hand_4,x_left_hand_5,x_left_hand_6,x_left_hand_7,...,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20,participant_id,phrase
928,1975562925,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.120838,-0.156574,-0.133080,-0.107317,-0.123403,-0.138411,-0.104443,-0.072335,181,"[39, 51, 51, 47, 50, 25, 14, 14, 54, 54, 54, 1..."
929,1975562925,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.051272,-0.066689,-0.055532,-0.043885,-0.069027,-0.072223,-0.055425,-0.040553,181,"[39, 51, 51, 47, 50, 25, 14, 14, 54, 54, 54, 1..."
930,1975562925,2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.041907,-0.052499,-0.042673,-0.033009,-0.058901,-0.057409,-0.040134,-0.025852,181,"[39, 51, 51, 47, 50, 25, 14, 14, 54, 54, 54, 1..."
931,1975562925,3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.125126,-0.159552,-0.143769,-0.124252,-0.122798,-0.134473,-0.109146,-0.085581,181,"[39, 51, 51, 47, 50, 25, 14, 14, 54, 54, 54, 1..."
932,1975562925,4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.129912,-0.168507,-0.150986,-0.128780,-0.130341,-0.151007,-0.125168,-0.098534,181,"[39, 51, 51, 47, 50, 25, 14, 14, 54, 54, 54, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,1975562925,431,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.174230,-0.225039,-0.232005,-0.235396,-0.155990,-0.180724,-0.177394,-0.176460,181,"[39, 51, 51, 47, 50, 25, 14, 14, 54, 54, 54, 1..."
1360,1975562925,432,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.185957,-0.238973,-0.245289,-0.248142,-0.164800,-0.190159,-0.186239,-0.184991,181,"[39, 51, 51, 47, 50, 25, 14, 14, 54, 54, 54, 1..."
1361,1975562925,433,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.179456,-0.229153,-0.238233,-0.241731,-0.166894,-0.196239,-0.195994,-0.194952,181,"[39, 51, 51, 47, 50, 25, 14, 14, 54, 54, 54, 1..."
1362,1975562925,434,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.154948,-0.193465,-0.187781,-0.180977,-0.150169,-0.164514,-0.151995,-0.143025,181,"[39, 51, 51, 47, 50, 25, 14, 14, 54, 54, 54, 1..."


In [3]:
# Save as csv to have a look if dataset is preprocessed like we wanted
df.to_csv('./dataset complete/test.csv')
#import os
#path = './dataset complete/test.csv'
#os.remove(path)

In [31]:
from copy import deepcopy
test_df = deepcopy(df)

In [40]:
with open ("./dataset complete/means_columns.json", "r") as f:
    mean_values = json.load(f)
mean_values

{'x_left_hand_0': 0.6971073150634766,
 'x_left_hand_1': 0.6097241640090942,
 'x_left_hand_2': 0.5367973446846008,
 'x_left_hand_3': 0.4998524487018585,
 'x_left_hand_4': 0.48945990204811096,
 'x_left_hand_5': 0.5789158344268799,
 'x_left_hand_6': 0.5327205061912537,
 'x_left_hand_7': 0.5006852149963379,
 'x_left_hand_8': 0.48087459802627563,
 'x_left_hand_9': 0.6335292458534241,
 'x_left_hand_10': 0.5850366353988647,
 'x_left_hand_11': 0.5553675889968872,
 'x_left_hand_12': 0.5426617860794067,
 'x_left_hand_13': 0.6885859966278076,
 'x_left_hand_14': 0.6425691843032837,
 'x_left_hand_15': 0.6139603853225708,
 'x_left_hand_16': 0.6030905246734619,
 'x_left_hand_17': 0.740994393825531,
 'x_left_hand_18': 0.7045613527297974,
 'x_left_hand_19': 0.6803600192070007,
 'x_left_hand_20': 0.6702653765678406,
 'x_pose_11': 0.8996241688728333,
 'x_pose_12': 0.2523534595966339,
 'x_pose_13': 1.096893072128296,
 'x_pose_14': 0.027202386409044266,
 'x_pose_15': 1.0204228162765503,
 'x_pose_16': 0.177

In [32]:
#198058957

test_df = test_df[test_df['sequence_id'] == 198041899]
test_df

,sequence_id,frame,x_left_hand_0,x_left_hand_1,x_left_hand_2,x_left_hand_3,x_left_hand_4,x_left_hand_5,x_left_hand_6,x_left_hand_7,...,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20,participant_id,phrase
169,198041899,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.138985,-0.212760,-0.220990,-0.218000,-0.174414,-0.228439,-0.229830,-0.224050,115,"[24, 18, 18, 20, 0, 54, 32, 43, 51, 46, 45, 40..."
170,198041899,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115,"[24, 18, 18, 20, 0, 54, 32, 43, 51, 46, 45, 40..."
171,198041899,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115,"[24, 18, 18, 20, 0, 54, 32, 43, 51, 46, 45, 40..."
172,198041899,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115,"[24, 18, 18, 20, 0, 54, 32, 43, 51, 46, 45, 40..."
173,198041899,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115,"[24, 18, 18, 20, 0, 54, 32, 43, 51, 46, 45, 40..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,198041899,119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.088853,-0.175088,-0.186027,-0.176915,-0.094460,-0.166891,-0.171645,-0.161031,115,"[24, 18, 18, 20, 0, 54, 32, 43, 51, 46, 45, 40..."
289,198041899,120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.083508,-0.188319,-0.195731,-0.176546,-0.100546,-0.177979,-0.174628,-0.153972,115,"[24, 18, 18, 20, 0, 54, 32, 43, 51, 46, 45, 40..."
290,198041899,121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.058275,-0.149976,-0.145411,-0.113003,-0.079266,-0.143865,-0.139109,-0.114143,115,"[24, 18, 18, 20, 0, 54, 32, 43, 51, 46, 45, 40..."
291,198041899,122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.063701,-0.149068,-0.151359,-0.124669,-0.087698,-0.144384,-0.138751,-0.114725,115,"[24, 18, 18, 20, 0, 54, 32, 43, 51, 46, 45, 40..."


In [25]:
import pandas as pd
import random

df_test = pd.read_csv('./dataset complete/train.csv')

p_ids = df_test.participant_id.unique().tolist()
len(p_ids)


TRAIN_SPLIT = 0.8
TEST_SPLIT = 0.2

num_train_samples = int(TRAIN_SPLIT * len(p_ids))

train_participants = random.sample(p_ids, num_train_samples)
test_participants = [x for x in p_ids if x not in train_participants]
print(train_participants)
print(test_participants)

[157, 63, 113, 236, 176, 38, 171, 21, 109, 117, 56, 6, 143, 225, 80, 186, 135, 202, 136, 121, 105, 95, 145, 122, 102, 223, 151, 27, 107, 88, 227, 203, 128, 40, 9, 241, 20, 89, 70, 158, 13, 239, 178, 187, 73, 168, 160, 92, 181, 231, 24, 233, 74, 169, 4, 125, 76, 254, 25, 138, 216, 230, 242, 217, 154, 43, 33, 161, 18, 147, 141, 72, 15, 196, 68]
[1, 93, 153, 246, 71, 2, 0, 59, 115, 36, 188, 81, 192, 251, 53, 112, 10, 219, 159]


In [26]:
test_participant_ids = [1, 38, 202, 71, 74, 102, 109, 125, 225, 161, 242, 59, 36, 188, 154, 80, 113, 88, 68]
train_participant_ids = [73, 95, 147, 18, 115, 216, 168, 56, 21, 159, 141, 143, 233, 181, 236, 9, 196, 53, 254, 251, 157, 20, 203, 112, 15, 72, 27, 178, 6, 153, 92, 169, 10, 145, 24, 192, 230, 186, 2, 135, 246, 43, 93, 33, 223, 81, 160, 231, 219, 171, 136, 25, 40, 176, 158, 121, 63, 105, 4, 128, 89, 13, 70, 138, 187, 241, 217, 107, 227, 122, 151, 117, 239, 0, 76]

In [18]:
def split_files(df: pd.DataFrame):
    global file_counter
    file_counter = 0
    phrases = df.sequence_id.unique()
    for i in phrases:

        phrase_table = df[df['sequence_id'] == i]
        #df_concat = pd.concat([df_concat, phrase_table], axis = 0)
        file_counter += 1

        if phrase_table.participant_id.unique()[0] in test_participant_ids:
            file_path = './dataset complete/preprocessed_files_data_generator/test_with_ones/' + str(file_counter) + '.pkl'
            phrase_table.to_pickle(file_path)
        else:
            file_path = './dataset complete/preprocessed_files_data_generator/train_with_ones/' + str(file_counter) + '.pkl'
            phrase_table.to_pickle(file_path)

In [45]:
# Preprocess supplemenatl data
import pandas as pd
import pyarrow.parquet as pq
import os
import json
import numpy as np
import pickle

file_names = os.listdir('./dataset complete/preprocessed_files/')
for i in range(0, len(file_names)):
    file_names[i] = file_names[i].replace('.parquet', '')

file_names

68

In [43]:
with open('./dataset complete/preprocessed_files_data_generator/supp_train_files/1.pkl', 'rb') as file:
    df = pickle.load(file)
df

,sequence_id,frame,x_left_hand_0,x_left_hand_1,x_left_hand_2,x_left_hand_3,x_left_hand_4,x_left_hand_5,x_left_hand_6,x_left_hand_7,...,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20,participant_id,phrase
14,1617965404,0,0.697107,0.609724,0.536797,0.499852,0.489460,0.578916,0.532721,0.500685,...,-0.179770,-0.256494,-0.242231,-0.220745,-0.199080,-0.243315,-0.227379,-0.207144,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
15,1617965404,1,0.021543,0.026809,0.024587,0.017511,0.010647,0.018573,0.006416,0.000562,...,0.045882,0.064743,0.065258,0.063156,0.051382,0.060955,0.058162,0.053934,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
16,1617965404,2,0.021543,0.026809,0.024587,0.017511,0.010647,0.018573,0.006416,0.000562,...,-0.013434,-0.019242,-0.015913,-0.012145,-0.014010,-0.019487,-0.017762,-0.015335,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
17,1617965404,3,0.021543,0.026809,0.024587,0.017511,0.010647,0.018573,0.006416,0.000562,...,-0.013434,-0.019242,-0.015913,-0.012145,-0.014010,-0.019487,-0.017762,-0.015335,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
18,1617965404,4,0.021543,0.026809,0.024587,0.017511,0.010647,0.018573,0.006416,0.000562,...,-0.013434,-0.019242,-0.015913,-0.012145,-0.014010,-0.019487,-0.017762,-0.015335,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,1617965404,184,-0.006822,-0.040116,-0.046174,-0.035516,-0.022779,-0.057302,-0.042873,-0.040654,...,0.012961,0.013156,0.014812,0.017274,-0.011088,-0.015729,-0.013964,-0.010203,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
199,1617965404,185,-0.006822,-0.040116,-0.046174,-0.035516,-0.022779,-0.057302,-0.042873,-0.040654,...,0.012961,0.013156,0.014812,0.017274,-0.011088,-0.015729,-0.013964,-0.010203,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
200,1617965404,186,-0.028941,-0.026870,-0.034222,-0.019173,0.012250,-0.000368,0.003322,0.001642,...,0.051892,0.060238,0.061862,0.064829,0.056684,0.059094,0.059267,0.060919,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."
201,1617965404,187,0.024804,0.024444,0.036103,0.024814,-0.001354,0.006331,-0.002192,-0.005577,...,-0.055890,-0.066385,-0.064825,-0.065623,-0.059703,-0.064367,-0.062722,-0.062456,31,"[51, 39, 36, 0, 47, 36, 45, 0, 40, 50, 0, 44, ..."


In [46]:
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

for fileNr in tqdm(file_names):
    # read in first parquet file
    with open('./dataset complete/preprocessed_files/' + fileNr, 'rb') as file:
        df = pickle.load(file)
    split_files(df)

  0%|          | 0/68 [00:00<?, ?it/s]


OSError: Cannot save file into a non-existent directory: 'dataset complete\preprocessed_files_data_generator\train_with_ones'

In [30]:
result = df_supp_ids.isin(test_participant_ids)
result


,IDs
0,False
1,False
2,False
3,True
4,False
...,...
67,False
68,False
69,False
70,False


In [31]:
result.to_csv('./dataset complete/test.csv')

In [41]:
#trivial interpolation function
INTERPOLATION_NAN_GAP = 15
FILL_NAN_VALUE = -1
SKIP_VALUE = 10000000
pd.options.mode.chained_assignment = None
file_counter = 0

def interpolate_dataset(df: pd.DataFrame):
    global file_counter
    phrases = df.sequence_id.unique()
    df_concat = pd.DataFrame()
    # each phrase
    for i in phrases:
        phrase_table = df[df['sequence_id'] == i]

        nan_count_left_hand = phrase_table['x_left_hand_0'].isna().sum()
        #print("NAN Count left: ", nan_count_left_hand)
        nan_count_right_hand = phrase_table['x_right_hand_0'].isna().sum()
        #print("NAN Count right: ", nan_count_right_hand)
        length_table = len(phrase_table)

        if nan_count_left_hand > 0.75*length_table and nan_count_right_hand > 0.75*length_table:
            print('Skipped')
            continue
        if length_table < INTERPOLATION_NAN_GAP:
            print('Skipped')
            continue

        # interpolation of N long nan gaps
        for column in phrase_table.columns.tolist():
            if(column == 'sequence_id') or (column == 'frame') or (column == 'participant_id') or (column == 'phrase'):
                continue
            if np.isnan(phrase_table.iloc[0][column]):
                phrase_table.at[phrase_table.index[0], column] = mean_values[column]

            nan_count = phrase_table[column].isnull().astype(int).groupby(phrase_table[column].notnull().cumsum()).cumsum()
            #nan_count.to_csv('./dataset complete/test.csv')
            #tmp = the number of nan values which is equal to the last next where next is unequal to 0
            tmp = 0
            for x in range(phrase_table.index[0],phrase_table.index[0] + len(nan_count)-1):
                curr = nan_count[x]
                next = nan_count[x+1]
                if next > curr:
                    tmp = next
                    if (x == phrase_table.index[0] + len(nan_count)-2):
                        for j in range(((x+2)-tmp), x+2):
                            nan_count[j] = SKIP_VALUE
                elif next == curr:
                    tmp = 0
                    continue
                elif (next < curr):
                    if tmp > INTERPOLATION_NAN_GAP:
                        for j in range(x-tmp, x+1):
                            nan_count[j] = SKIP_VALUE
                    tmp = 0
            # should be in right order, but to be sure
            nan_count = nan_count.sort_index()
            phrase_table[column] = phrase_table[column].interpolate() \
                .where(nan_count < SKIP_VALUE, np.nan)

        #df_concat = pd.concat([df_concat, phrase_table], axis = 0)
        right_pose_columns = ['12', '14', '16']
        left_pose_columns = ['11', '13', '15']

        # replace missing hand nans
        phrase_table['marking_left_replace'] = 0
        phrase_table['marking_right_replace'] = 0
        for i, row in phrase_table.iterrows():
            if np.isnan(row['x_left_hand_0']) and not np.isnan(row['x_right_hand_0']):
                row['marking_left_replace'] = 1
                for column in phrase_table.columns.tolist():
                    if 'left' in column:
                        corresponding_column = column.replace('left', 'right')
                        if 'z' in column:
                            phrase_table.at[i, column] = row[corresponding_column]
                        else:
                            phrase_table.at[i, column] = 1 - row[corresponding_column]
                    if 'pose' in column and column[-2:] in left_pose_columns:
                        corresponding_column = column[:-2] + str(int(column[-2:]) + 1)
                        if 'z' in column:
                            phrase_table.at[i, column] = row[corresponding_column]
                        else:
                            phrase_table.at[i, column] = 1 - row[corresponding_column]
            if np.isnan(row['x_right_hand_0']) and not np.isnan(row['x_left_hand_0']):
                row['marking_right_replace'] = 1
                for column in phrase_table.columns.tolist():
                    if 'right' in column:
                        corresponding_column = column.replace('right', 'left')
                        if 'z' in column:
                            phrase_table.at[i, column] = row[corresponding_column]
                        else:
                            phrase_table.at[i, column] = 1 - row[corresponding_column]
                    if 'pose' in column and column[-2:] in right_pose_columns:
                        corresponding_column = column[:-2] + str(int(column[-2:]) - 1)
                        if 'z' in column:
                            phrase_table.at[i, column] = row[corresponding_column]
                        else:
                            phrase_table.at[i, column] = 1 - row[corresponding_column]


        row_indices_filled_after = phrase_table[(phrase_table['marking_left_replace'] == 0) & (phrase_table.shift(-1)['marking_left_replace'] == 1)].index.tolist()
        for r_idx in row_indices_filled_after:
            for column in phrase_table.columns.tolist():
                if 'left' in column:
                    end_idx = min(r_idx + INTERPOLATION_NAN_GAP // 2, len(phrase_table) - 1 + phrase_table.index[0])
                    phrase_table.loc[r_idx:end_idx, column] = np.nan
                if 'pose' in column and column[-2:] in left_pose_columns:
                    end_idx = min(r_idx + INTERPOLATION_NAN_GAP // 2, len(phrase_table) - 1 + phrase_table.index[0])
                    phrase_table.loc[r_idx:end_idx, column] = np.nan
        row_indices_filled_after = phrase_table[(phrase_table['marking_right_replace'] == 0) & (phrase_table.shift(-1)['marking_right_replace'] == 1)].index.tolist()
        for r_idx in row_indices_filled_after:
            for column in phrase_table.columns.tolist():
                if 'right' in column:
                    end_idx = min(r_idx + INTERPOLATION_NAN_GAP // 2, len(phrase_table) - 1 + phrase_table.index[0])
                    phrase_table.loc[r_idx:end_idx, column] = np.nan
                if 'pose' in column and column[-2:] in right_pose_columns:
                    end_idx = min(r_idx + INTERPOLATION_NAN_GAP // 2, len(phrase_table) - 1 + phrase_table.index[0])
                    phrase_table.loc[r_idx:end_idx, column] = np.nan
        row_indices_filled_before = phrase_table[(phrase_table['marking_right_replace'] == 0) & (phrase_table.shift(1)['marking_right_replace'] == 1)].index.tolist()
        for r_idx in row_indices_filled_before:
            for column in phrase_table.columns.tolist():
                if 'right' in column:
                    start_idx = max(phrase_table.index[0]+1, r_idx-INTERPOLATION_NAN_GAP//2)
                    phrase_table.loc[start_idx:r_idx, column] = np.nan
                if 'pose' in column and column[-2:] in right_pose_columns:
                    start_idx = max(phrase_table.index[0]+1, r_idx-INTERPOLATION_NAN_GAP//2)
                    phrase_table.loc[start_idx:r_idx, column] = np.nan
        row_indices_filled_before = phrase_table[(phrase_table['marking_left_replace'] == 0) & (phrase_table.shift(1)['marking_left_replace'] == 1)].index.tolist()
        for r_idx in row_indices_filled_before:
            for column in phrase_table.columns.tolist():
                if 'left' in column:
                    start_idx = max(phrase_table.index[0]+1, r_idx-(INTERPOLATION_NAN_GAP//2))
                    phrase_table.loc[start_idx:r_idx, column] = np.nan
                if 'pose' in column and column[-2:] in left_pose_columns:
                    start_idx = max(phrase_table.index[0]+1, r_idx-INTERPOLATION_NAN_GAP//2)
                    phrase_table.loc[start_idx:r_idx, column] = np.nan

        phrase_table = phrase_table.drop('marking_left_replace', axis=1)
        phrase_table = phrase_table.drop('marking_right_replace', axis=1)

        for column in phrase_table.columns.tolist():
            if(column == 'sequence_id') or (column == 'frame') or (column == 'participant_id') or (column == 'phrase'):
                continue
            if np.isnan(phrase_table.iloc[0][column]):
                phrase_table.at[phrase_table.index[0], column] = mean_values[column]

        # replace with previous value
        phrase_table['keep'] = 0
        after_nan_indices = phrase_table[(~phrase_table.isna()) & (phrase_table.shift(1).isna())].index.tolist()
        for r_idx in after_nan_indices:
            phrase_table.loc[max(phrase_table.index[1], r_idx-INTERPOLATION_NAN_GAP//2):r_idx, 'keep'] = 1
        #phrase_table = phrase_table.fillna(method='ffill').where(phrase_table['keep'] == 0, np.nan)

        phrase_table.loc[phrase_table['keep'] != 1] = phrase_table.loc[phrase_table['keep'] != 1].fillna(method='ffill')

        phrase_table = phrase_table.drop('keep', axis=1)


        # linear interpolation
        phrase_table = phrase_table.interpolate(method='linear')

        # check if all nans have been removed
        if len(phrase_table[phrase_table.isna().any(axis=1)]) != 0:
            print("Still NANs!")


        # use relative values
        for column in phrase_table.columns.tolist():
            if(column == 'sequence_id') or (column == 'frame') or (column == 'participant_id') or (column == 'phrase'):
                continue
            first_value = phrase_table[column].loc[phrase_table.index[0]]
            phrase_table[column] = phrase_table[column] - phrase_table[column].shift(1)
            phrase_table.at[phrase_table.index[0], column] = first_value



        #df_concat = pd.concat([df_concat, phrase_table], axis = 0)
        file_counter += 1

        if phrase_table.participant_id.unique()[0] in test_participant_ids:
            file_path = './dataset complete/preprocessed_files_data_generator/supp_test_files/' + str(file_counter) + '.pkl'
            phrase_table.to_pickle(file_path)
        else:
            file_path = './dataset complete/preprocessed_files_data_generator/supp_train_files/' + str(file_counter) + '.pkl'
            phrase_table.to_pickle(file_path)

        #phrase_table.to_csv('./dataset complete/test.csv')
    #df_concat = df_concat.fillna(FILL_NAN_VALUE)
    #return df_concat

In [79]:
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

interpolated_df = interpolate_dataset(test_df)

In [82]:
import pandas as pd
import pyarrow.parquet as pq
import os
import json
import numpy as np
import pickle

file_names = os.listdir('./dataset complete/train_landmarks/')
for i in range(0, len(file_names)):
    file_names[i] = file_names[i].replace('.parquet', '')

file_names

['1019715464',
 '1021040628',
 '105143404',
 '1098899348',
 '1099408314',
 '1133664520',
 '1134756332',
 '1255240050',
 '128822441',
 '1320204318',
 '1341528257',
 '1358493307',
 '1365275733',
 '1365772051',
 '1405046009',
 '1448136004',
 '1497621680',
 '149822653',
 '152029243',
 '1552432300',
 '1557244878',
 '1562234637',
 '1643479812',
 '1647220008',
 '1662742697',
 '1664666588',
 '169560558',
 '1726141437',
 '175396851',
 '1785039512',
 '1865557033',
 '1880177496',
 '1905462118',
 '1906357076',
 '1920330615',
 '1967755728',
 '1969985709',
 '1997878546',
 '2026717426',
 '2036580525',
 '2072296290',
 '2072876091',
 '2118949241',
 '234418913',
 '296317215',
 '349393104',
 '388576474',
 '425182931',
 '433948159',
 '450474571',
 '474255203',
 '495378749',
 '522550314',
 '527708222',
 '532011803',
 '5414471',
 '546816846',
 '566963657',
 '568753759',
 '614661748',
 '638508439',
 '649779897',
 '654436541',
 '683666742',
 '871280215',
 '882979387',
 '933868835',
 '939623093']

In [36]:
# Preprocess supplemenatl data
supp_train = pd.read_csv('./dataset complete/supplemental_metadata.csv')

with open ("./dataset complete/character_to_prediction_index.json", "r") as f:
    characters = json.load(f)

supp_train = exchange_string_with_numbers_array(supp_train, characters)
supp_train

,sequence_id,participant_id,phrase
0,1535467051,251,"[34, 46, 44, 40, 45, 38, 0, 52, 47, 0, 54, 40,..."
1,1535499058,239,"[54, 36, 0, 33, 36, 51, 51, 36, 49, 0, 40, 45,..."
2,1535530550,245,"[40, 45, 51, 36, 49, 36, 50, 51, 40, 45, 38, 0..."
3,1535545499,38,"[53, 40, 34, 51, 40, 44, 50, 0, 35, 36, 50, 36..."
4,1535585216,254,"[42, 45, 36, 36, 0, 33, 46, 45, 36, 0, 40, 50,..."
...,...,...,...
52953,1090866442,239,"[54, 32, 45, 51, 0, 51, 46, 0, 41, 46, 40, 45,..."
52954,1090966452,95,"[51, 39, 40, 50, 0, 47, 39, 36, 45, 46, 44, 36..."
52955,1091005846,40,"[51, 39, 36, 0, 54, 40, 45, 45, 36, 49, 0, 46,..."
52956,1091011550,241,"[32, 49, 36, 0, 56, 46, 52, 0, 50, 52, 49, 36,..."


In [37]:
# Preprocess supplemenatl data
import pandas as pd
import pyarrow.parquet as pq
import os
import json
import numpy as np
import pickle

file_names = os.listdir('./dataset complete/supplemental_landmarks/')
for i in range(0, len(file_names)):
    file_names[i] = file_names[i].replace('.parquet', '')

file_names

['1032110484',
 '1047404576',
 '1112747136',
 '1118603411',
 '1144115867',
 '1176508147',
 '1249944812',
 '1279694894',
 '131312512',
 '1407656790',
 '1471096258',
 '1471341722',
 '1505488209',
 '1579345709',
 '1624527344',
 '1650637630',
 '1682915129',
 '1727438550',
 '1755047076',
 '1756773911',
 '1779786322',
 '1857374937',
 '1881515495',
 '193950599',
 '2057261717',
 '2100073719',
 '236903981',
 '285528514',
 '293101677',
 '333606065',
 '33432165',
 '369584223',
 '371169664',
 '440362090',
 '442061898',
 '471766624',
 '595441814',
 '597469033',
 '636900267',
 '639454452',
 '676340265',
 '680303484',
 '697480828',
 '716508881',
 '736978972',
 '756566775',
 '775880548',
 '778903889',
 '86446671',
 '897287709',
 '924144755',
 '95345213',
 '971104021']

In [42]:
# Preprocess supplemenatl data
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

for file in tqdm(file_names):

    # read in first parquet file
    supp_landmarks = pq.read_table('./dataset complete/supplemental_landmarks/' + file + '.parquet')
    supp_landmarks = supp_landmarks.to_pandas()

    # leave only columns with hand and arm data
    supp_landmarks = remove_unwanted_columns(supp_landmarks)
    # merge the hand and arm data with the corresponding participant id and phrase
    df = merge_datasets(supp_landmarks, supp_train)
    # for short nan gaps interpolate the values, else fill Nans with a placeholder number
    interpolate_dataset(df)

  0%|          | 0/53 [00:00<?, ?it/s]

Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped


  2%|▏         | 1/53 [07:54<6:50:51, 474.06s/it]

Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped


  4%|▍         | 2/53 [15:30<6:34:21, 463.95s/it]

...

 96%|█████████▌| 51/53 [6:39:43<15:44, 472.06s/it]

Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped


 98%|█████████▊| 52/53 [6:47:38<07:52, 472.99s/it]

Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped


100%|██████████| 53/53 [6:55:28<00:00, 470.35s/it]


In [83]:
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

for file in tqdm(file_names):

    # read in first parquet file
    train_landmarks = pq.read_table('./dataset complete/train_landmarks/' + file + '.parquet')
    train_landmarks = train_landmarks.to_pandas()

    # leave only columns with hand and arm data
    train_landmarks = remove_unwanted_columns(train_landmarks)
    # merge the hand and arm data with the corresponding participant id and phrase
    df = merge_datasets(train_landmarks, df_train)
    # for short nan gaps interpolate the values, else fill Nans with a placeholder number
    interpolate_dataset(df)


  0%|          | 0/68 [00:00<?, ?it/s]

Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped


  1%|▏         | 1/68 [07:20<8:12:20, 440.90s/it]

Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped


  3%|▎         | 2/68 [14:47<8:08:27, 444.05s/it]

...

 97%|█████████▋| 66/68 [8:05:10<14:39, 439.66s/it]

Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped


 99%|█████████▊| 67/68 [8:12:26<07:18, 438.61s/it]

Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped


100%|██████████| 68/68 [8:19:50<00:00, 441.03s/it]


In [267]:
interpolated_df = interpolate_dataset(test_df)

C:\Users\WG\AppData\Local\Temp\ipykernel_2240\169649542.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  phrase_table['marking_left_replace'] = 0
C:\Users\WG\AppData\Local\Temp\ipykernel_2240\169649542.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  phrase_table['marking_right_replace'] = 0


Still NANs!


In [236]:
interpolated_df.to_csv('./dataset complete/test.csv')